In [1]:
import pandas as pd
import seaborn as srn
import statistics as sts

In [3]:
dataset = pd.read_csv('Churn.csv', sep=';')
dataset

,X0,X1,X2,X3,X4,X4.1,X6,X7,X8,X9,X10,X11
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
994,996,838,SC,Masculino,43,9,12310588,2,1,0,14576583.0,0
995,997,610,SC,Masculino,29,9,0,3,0,1,8391224.0,0
996,998,811,SC,Masculino,44,3,0,2,0,1,7843973.0,0
997,999,587,SC,Masculino,62,7,12128627,1,0,1,677692.0,0


In [4]:
dataset.columns = ['Id', 'Score', 'Estado', 'Genero', 'Idade', 'Patrimonio', 'Saldo', 'Produtos', 'TemCartCredito',
                  'Ativo', 'Salario', 'Saiu']
dataset.head()

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [6]:
# Tratar a coluna 'Salario', que possui NaN. Os Nan serão substituídos pela mediana.

dataset['Salario'].describe()

count    9.920000e+02
mean     3.528762e+07
std      5.305800e+08
min      9.677000e+03
25%      3.029011e+06
50%      8.703250e+06
75%      1.405213e+07
max      1.193469e+10
Name: Salario, dtype: float64

In [7]:
# Substituir os NaN pela mediana dos valores da coluna.

In [9]:
# Calcular a mediana de 'Salario'.

mediana = sts.median(dataset['Salario'])
mediana

70518.0

In [10]:
# Substituir NaN de 'Salario' pela mediana dos valores da coluna.

dataset['Salario'].fillna(mediana, inplace=True) 
# inplace vai substituir no próprio conjunto de dados, não vai retornar um objeto com o valor substituído.

In [11]:
# Verificar se ainda existem NaN em 'Salario'.

dataset['Salario'].isnull().sum()

0

In [12]:
# Tratar a coluna 'Genero', que possui valores inconsistentes e NaN. Os NaN serão substituídos pela moda.

agrupado = dataset.groupby(['Genero']).size()
agrupado

Genero
F              2
Fem            1
Feminino     461
M              6
Masculino    521
dtype: int64

In [14]:
# Tratar a inconsistência dos dados de 'Genero'. Padronizar de acordo com o domínio.

dataset.loc[dataset['Genero'] == 'M', 'Genero'] = 'Masculino'
dataset.loc[dataset['Genero'].isin(['Fem', 'F']), 'Genero'] = 'Feminino'

In [15]:
agrupado = dataset.groupby(['Genero']).size()
agrupado

Genero
Feminino     464
Masculino    527
dtype: int64

In [16]:
# Substituir os NaN de 'Genero' pela moda dos valores da coluna.

# Obter a moda
moda = sts.mode(dataset['Genero'])
moda

'Masculino'

In [17]:
# Substituir pela moda.

dataset['Genero'].fillna(moda, inplace=True)

In [18]:
# Verificar se ainda existem NaN em 'Genero'.

dataset['Genero'].isnull().sum()

0

In [20]:
# Tratar a coluna 'Idade', que possui valores fora do domínio. Estes valores serão substituídos pela mediana.

dataset['Idade'].describe()

count    999.000000
mean      38.902903
std       11.401912
min      -20.000000
25%       32.000000
50%       37.000000
75%       44.000000
max      140.000000
Name: Idade, dtype: float64

In [22]:
# Para selecionar os dados de idade que estão fora do domínio e visualizá-los.

dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
867,869,636,RS,Feminino,-10,1,17083346,1,1,0,11051028.0,1
984,986,773,RS,Masculino,-20,1,12453278,2,0,1,1172357.0,0
990,992,655,RS,Masculino,140,5,93147,2,1,0,6621413.0,0


In [23]:
# Obter a mediana.

mediana = sts.median(dataset['Idade'])
mediana

37

In [24]:
# Substituir os valores fora de domínio pela mediana.

dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120), 'Idade'] = mediana

In [25]:
# Verificar se ainda existem valores fora de domínio.

dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu


In [26]:
# Tratar registros duplicados. Excluir um registro.

# Para encontrar registros duplicados, verificar se há 'Id' duplicado.

dataset[dataset.duplicated(['Id'], keep=False)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
80,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0
81,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0


In [27]:
# Excluir pelo 'Id'

dataset.drop_duplicates(subset='Id', keep='first', inplace=True)
# Métodos do próprio objeto dataframe precisam do parâmetro inplace=True para que as alterações sejam persistidas.

In [29]:
# Verificar se ainda há registros duplicados.

dataset[dataset.duplicated(['Id'], keep=False)]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu


In [30]:
# Tratar valores de 'Estado' fora do domínio. Substituir esses valores pela moda.

agrupado = dataset.groupby(['Estado']).size()
agrupado

Estado
PR    257
RP      1
RS    477
SC    258
SP      4
TD      1
dtype: int64

In [31]:
# Encontrar a moda.

moda = sts.mode(dataset['Estado'])
moda

'RS'

In [32]:
# Substituir os valores fora de domínio pela moda.

dataset.loc[dataset['Estado'].isin(['RP', 'SP', 'TD']), 'Estado'] = moda

In [33]:
# Verificar se ainda há valores fora de domínio.

agrupado = dataset.groupby(['Estado']).size()
agrupado

Estado
PR    257
RS    483
SC    258
dtype: int64

In [34]:
# Tratar os outliers de 'Salario'. Outliers serão os valores maiores que 2x desvio padrão.
# Valores serão substituídos pela mediana.

# Calcular o desvio padrão.

desv = sts.stdev(dataset['Salario'])
desv

528988918.4679201

In [36]:
# Selecionar os outliers de 'Salario'.

dataset.loc[dataset['Salario'] > 2*desv]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
7,8,376,PR,Feminino,29,4,11504674,4,1,0,1.193469e+10,1
116,118,668,PR,Feminino,37,6,1678644,1,1,0,1.156383e+10,0
170,172,484,RS,Feminino,29,4,13011439,1,1,0,1.640179e+09,0
230,232,673,RS,Masculino,72,1,0,2,0,1,1.119812e+09,0


In [37]:
# Calcular a mediana de 'Salario'.

mediana = sts.median(dataset['Salario'])
mediana

8637195.5

In [39]:
# Substituir os outliers pela mediana.

dataset.loc[dataset['Salario'] > 2*desv, 'Salario'] = mediana

In [40]:
# Verificar se ainda há outliers.

dataset.loc[dataset['Salario'] > 2*desv]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu


In [41]:
dataset.head()

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [42]:
dataset.shape

(998, 12)